In [1]:
import sys
sys.path.append("../../")

import os
os.environ["BASE_PATH"] = "../.."

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import torch
import pandas as pd

In [4]:
from rfi_class.preprocessing.create_dataset import create_dataset, create_folds
from rfi_class.data_loader.data_loader import create_data_loader

In [5]:
create_dataset()
create_folds()

In [6]:
df = pd.read_csv("../../data/train_folds.csv")

In [7]:
fold = 0

In [8]:
df_train = df[df.kfold != fold].reset_index(drop=True)
df_test = df[df.kfold == fold].reset_index(drop=True)

In [9]:
df_train.shape

(3072, 4)

In [10]:
df_test.shape

(768, 4)

In [11]:
batch_size = 2
train_loader = create_data_loader(df, batch_size=batch_size)

In [12]:
# for batch in train_loader:
#     print(batch)
#     break

In [13]:
# batch["image"][0]

In [14]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(npimg)
    plt.show()

In [15]:
# imshow(batch["image"][1])

In [16]:
from torch import nn
import torch.nn.functional as F


class RFINet(nn.Module):
    def __init__(self, pool_size=(4, 4)):
        super(RFINet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.maxpool = nn.AdaptiveMaxPool2d(pool_size)
        self.fc1 = nn.Linear(64*pool_size[0]*pool_size[1], 500)
        self.fc2 = nn.Linear(500, 4)
   
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [17]:
net = RFINet()


In [18]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [19]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        images= batch["image"]
        images = images.permute(0, 3, 1,2)
        
        targets= batch["target"]
        targets = targets.type(torch.LongTensor)
#         print(images.shape)
#         print(images.dtype)
#         print(targets.shape)
#         print(targets.dtype)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.090
[1,    20] loss: 0.007
[1,    30] loss: 0.007
[1,    40] loss: 0.007
[1,    50] loss: 0.007
[1,    60] loss: 0.007


Traceback (most recent call last):
  File "/Users/kunalgusain/miniconda3/envs/kaggle/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/Users/kunalgusain/miniconda3/envs/kaggle/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/kunalgusain/miniconda3/envs/kaggle/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/kunalgusain/miniconda3/envs/kaggle/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 